In [ ]:
!csEvalPixelLevelSemanticLabeling
!pip install segmentation-models-pytorch
!pip install cityscapesscripts
!python -m pip install cityscapesscripts[gui]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
import os
import sys

GOOGLE_DRIVE_PATH = "path/to/your/drive/folder"
sys.path.append(GOOGLE_DRIVE_PATH)

In [3]:
import glob
import numpy as np
import torch
import cv2
from torchvision.datasets.vision import VisionDataset
from torch.utils.data import DataLoader
from PIL import Image

In [4]:
class CityscapesDataset(VisionDataset):
    def __init__(self, root, split='train', preprocessing=None, transform=None, target_transform=None):
        super(CityscapesDataset, self).__init__(root, transform=transform, target_transform=target_transform)
        self.preprocessing = preprocessing
        self.images = sorted(glob.glob(os.path.join(root, 'leftImg8bit', split, '**', '*_leftImg8bit.png'), recursive=True))
        self.targets = sorted(glob.glob(os.path.join(root, 'gtFine', split, '**', '*_gtFine_labelTrainIds.png'), recursive=True))

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        img_path = self.images[index]
        label_path = self.targets[index]

        image = Image.open(img_path).convert('RGB')
        label = Image.open(label_path).convert('L')


        if self.transform:
            image = self.transform(image)

        if self.preprocessing:
            image = self.preprocessing(np.array(image))
            image = torch.tensor(image).permute(2, 0, 1).float()

        if self.target_transform:
            label = self.target_transform(label)
        else:
            label = torch.tensor(np.array(label), dtype=torch.long)

        return image, label

In [5]:
import segmentation_models_pytorch as smp

preprocessing_fn = smp.encoders.get_preprocessing_fn("resnet34", "imagenet")

In [6]:
dataset_path = os.path.join(GOOGLE_DRIVE_PATH, 'cityscapes')
val_dataset = CityscapesDataset(root=dataset_path, split='val', preprocessing=preprocessing_fn)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, num_workers=4)

Number of val images: 500
Number of val targets: 500
Example image shape: torch.Size([3, 1024, 2048])
Example label shape: torch.Size([1024, 2048])
Example label unique values: tensor([  0,   1,   2,   4,   5,   7,   8,  10,  12,  13,  18, 255])


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [7]:
predictions_path = os.path.join(GOOGLE_DRIVE_PATH,'predictions')

In [8]:
model_file_path = os.path.join(GOOGLE_DRIVE_PATH,'segmentation_best.pth')
model = torch.load(model_file_path)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [9]:
from cityscapesscripts.helpers.labels import labels

train_lbl_ids = {label.trainId: label.id for label in labels if label.trainId != 255}

def convert_to_label_id(pred):
    pred_id = pred.copy()
    for trainid, lbl_id in train_lbl_ids.items():
        pred_id[pred == trainid] = lbl_id
    return pred_id

with torch.no_grad():
    for idx, (images, labels) in enumerate(val_loader):
        images = images.to(device)
        outputs = model(images)
        preds = torch.argmax(outputs, dim=1).cpu().numpy()
        for i in range(images.size(0)):
            pred = preds[i]
            pred = convert_to_label_id(pred)
            pred_img = Image.fromarray(pred.astype(np.uint8))
            img_name = os.path.basename(val_dataset.images[idx * images.size(0) + i]).replace('_leftImg8bit.png', '_pred.png')
            pred_img.save(os.path.join(predictions_path, img_name))

In [10]:
os.environ['CITYSCAPES_RESULTS'] = predictions_path
os.environ['CITYSCAPES_DATASET'] = dataset_path
!echo $CITYSCAPES_RESULTS
!echo $CITYSCAPES_DATASET

drive/MyDrive/semester_6/computer_vision/homework4/predictions_fold3
drive/MyDrive/semester_6/computer_vision/homework4/cityscapes


In [11]:
!csEvalPixelLevelSemanticLabeling

Evaluating 500 pairs of images...
Images Processed: 500 

--------------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------- 
              |   u   |   e   |   r   |   o   |   s   |   d   |   g   |   r   |   s   |   p   |   r   |   b   |   w   |   f   |   g   |   b   |   t   |   p   |   p   |   t   |   t   |   v   |   t   |   s   |   p   |   r   |   c   |   t   |   b   |   c   |   t   |   t   |   m   |   b   |  Prior |
--------------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------